In [ ]:
import cv2
import ctypes

# DPI awareness code is Windows-specific and not available on Linux/WSL
# awareness = ctypes.c_int()
# from ctypes import windll
# windll.shcore.GetProcessDpiAwareness(0, ctypes.byref(awareness))
# print(awareness)

# img_path = "../images/Screenshot_20250720_131146_Nonogram_galaxy.png"
# img = cv2.imread(img_path)

ImportError: cannot import name 'windll' from 'ctypes' (/usr/lib/python3.13/ctypes/__init__.py)

In [3]:
def show_image(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)
edges = cv2.Canny(blur, 50, 150)
show_image("Edges", edges)

In [1]:
import cv2
import numpy as np

# Load your image
img = cv2.imread('../images/Screenshot_20250720_131146_Nonogram_galaxy.png')

# Scale factor for 4K display (adjust this value as needed)
scale_factor = 2.0  # 2x larger, try 3.0 or 4.0 if still too small

# Interactive edge detection with sliders
def update_edges(val=None):
    # Get trackbar values
    kernel_size = cv2.getTrackbarPos('Kernel Size', 'Interactive Edges')
    # Ensure kernel size is odd and at least 1
    if kernel_size % 2 == 0:
        kernel_size += 1
    if kernel_size < 1:
        kernel_size = 1
    
    sigma = cv2.getTrackbarPos('Sigma', 'Interactive Edges')
    low_threshold = cv2.getTrackbarPos('Low Threshold', 'Interactive Edges')
    high_threshold = cv2.getTrackbarPos('High Threshold', 'Interactive Edges')
    
    # Apply processing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), sigma)
    edges = cv2.Canny(blur, low_threshold, high_threshold)
    
    # Scale up the image for better visibility on 4K monitor
    height, width = edges.shape
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    edges_scaled = cv2.resize(edges, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
    
    # Display result
    cv2.imshow('Interactive Edges', edges_scaled)

# Create window
cv2.namedWindow('Interactive Edges', cv2.WINDOW_NORMAL)

# Create trackbars
cv2.createTrackbar('Kernel Size', 'Interactive Edges', 5, 21, update_edges)  # Blur kernel size (1-21, odd numbers work best)
cv2.createTrackbar('Sigma', 'Interactive Edges', 0, 10, update_edges)        # Blur sigma (0-10)
cv2.createTrackbar('Low Threshold', 'Interactive Edges', 50, 255, update_edges)   # Canny low threshold
cv2.createTrackbar('High Threshold', 'Interactive Edges', 120, 255, update_edges) # Canny high threshold

# Initial display
update_edges()

# Wait for key press and cleanup
cv2.waitKey(0)
cv2.destroyAllWindows()